# WorkShop 2

Let"s play with pandas and answer "stakeholders" _questions_.

Problems $2-13$ rely on your `Problem 1` answer.

## Problem  1

Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of __energy_df__.

Keep in mind that this is an _Excel file_, and not a _comma separated values file_. Also, make sure to _exclude_ the footer and header information from the datafile. The first _two columns_ are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

```python
['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]
```

Convert `Energy Supply` to _gigajoules_ (there are $1000000$ _gigajoules_ in a _petajoule_**). For all countries which have missing data (_e.g._ data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (_for use in later questions_):

```bash
"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"
```

There are also several countries with parenthesis in their name. Be sure to _remove these_, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with _Numeric digits_ in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a _csv_ containing countries' GDP from $1960$ to $2015$ from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame __gdp_df__. 

Make sure to _skip the header_, and _rename_ the following list of countries:

```bash
"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"
```

Finally, load the [Scimago Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their _journal contributions_ in the aforementioned area. Call this DataFrame __scimago_df__.

Join the _thrgoee datasets_: _energy\_df_, _gdp\_df_ and _scimago\_df_, into a new dataset (using the _intersection_ of _country names_). Use only the _last_ $10$ years ($2006-2015$) of __GDP data__ and only the _top 15 countries_ by __Scimagojr 'Rank'__ (_Rank 1 through 15_). 

The _index_ of this DataFrame should be the __name__ of the country, and the _columns_ should be:
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']`.

_This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank"._

In [55]:
# !pip install xlrd
# !pip install openpyxl
import re
import pandas as pd
import numpy as np

def generate_energy_df():
    # YOUR CODE HERE load dataframe and remove header and footer

    energy_df = pd.read_excel("assets/Energy Indicators.xls")

    energy_df = energy_df[17:244].reset_index()

    energy_df = energy_df.drop(columns=["index", "Unnamed: 0", "Unnamed: 1"])
    energy_df = energy_df.rename(columns={"Unnamed: 2":"Country", "Unnamed: 3":"Energy Supply", "Unnamed: 4":"Energy Supply per Capita", "Unnamed: 5":"% Renewable"})

    # YOUR CODE HERE transform data

    energy_df = energy_df.map(lambda data: np.nan if data == "..." else data)

    energy_df["Energy Supply"] = energy_df["Energy Supply"].apply(lambda x: x*1000000)

    # YOUR CODE HERE transform Country column
    energy_df["Country"] = energy_df["Country"].replace("Republic of Korea", "South Korea")
    energy_df["Country"] = energy_df["Country"].replace("United States of America20", "United States")
    energy_df["Country"] = energy_df["Country"].replace("United Kingdom of Great Britain and Northern Ireland19", "United Kingdom")
    energy_df["Country"] = energy_df["Country"].replace("China, Hong Kong Special Administrative Region3", "Hong Kong")
    energy_df["Country"] = energy_df["Country"].apply(lambda x: re.sub(r"\(.*\)*", "", x).rstrip())

    energy_df["Country"] = energy_df["Country"].apply(lambda x: re.sub(r"\d*", "", x))
    return energy_df

def get_GDP_df():
    def is_number(s: str) -> bool:
        """
        Check if a string is a number.

        Args:
            s (str): The string to check.

        Returns:
            A boolean indicating if the string is a number.
        """
        try:
            int(s)  
            return True
        except ValueError:
            return False

    # YOUR CODE HERE load data, remove header, adjust column names

    gdp_df = pd.read_csv("assets/world_bank.csv")
    gdp_df.columns = gdp_df.iloc[3]
    gdp_df = gdp_df[4:]
    gdp_df = gdp_df.reset_index()
    gdp_df = gdp_df.drop(columns=["index"])
    gdp_df.columns = gdp_df.columns.map(lambda row: str(int(row)) if is_number(row) else row)

    # YOUR CODE HERE transform Country column 

    gdp_df["Country Name"] = gdp_df["Country Name"].replace("Korea, Rep.","South Korea")
    gdp_df["Country Name"] = gdp_df["Country Name"].replace("Iran, Islamic Rep.","Iran")
    gdp_df["Country Name"] = gdp_df["Country Name"].replace("Hong Kong SAR, China","Hong Kong")

    return gdp_df

def get_scimago_df():
    # YOUR CODE HERE load data
    scimago_df = pd.read_excel("assets/scimagojr-3.xlsx")
    return scimago_df

def answer_one():
    # YOUR CODE HERE
    
    energy_df = generate_energy_df().set_index("Country")
    gdp_df = get_GDP_df().set_index("Country Name")
    scimago_df = get_scimago_df().set_index("Country")
    df = scimago_df.iloc[:15].join(energy_df, how="inner").join(gdp_df.iloc[:, 49:], how="inner")

    return df
# example of test cases
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"
assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"

In [56]:
# example of test cases
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"
assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"

In [57]:
# space for proffesor tests

## Problem 2

The _Problem 1_ joined three datasets then reduced this to just the top $15$ entries. When you joined the datasets, but before you reduced this to the top $15$ items, how many entries did you lose?

_This function should return a single number._

In [58]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [59]:
def answer_two():
    energy_df = generate_energy_df().set_index('Country').iloc[:15]
    gdp_df = get_GDP_df().set_index('Country Name').iloc[:15]
    scimago_df = get_scimago_df().set_index('Country').iloc[:15]
    
    df = scimago_df.join(energy_df, how="inner").join(gdp_df.iloc[:, 49:], how="inner")
    losed_entries = answer_one().shape[0] - df.shape[0]
    return losed_entries
# example of test cases
assert type(answer_two()) == int, "Q2: You should return an int number!"

In [60]:
# example of test cases
assert type(answer_two()) == int, "Q2: You should return an int number!"

In [61]:
# space for proffesor tests

## Problem 3

What are the top $15$ countries for average GDP over the last $10$ years?

_This function should return a Series named `avg_gdp` with $15$ countries and their average GDP sorted in descending order._

In [62]:
def answer_three():
    # YOUR CODE HERE
    avg_gdp = answer_one().iloc[:, 10:].mean(axis=1)
    avg_gdp = pd.Series(avg_gdp, name='avg_gdp')
    return avg_gdp.sort_values(ascending=False)
# example of test cases
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"

In [63]:
# example of test cases
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"

In [64]:
# space for proffesor tests

# Problem 4

By how much had the GDP changed over the $10$ year span for the country with the $6-th$ largest average GDP?

_This function should return a single number._

In [65]:
def answer_four():
    # YOUR CODE HERE

    rank_df = answer_one()

    country_6th_avg = rank_df.loc[answer_three().index[5]]
    change = float(country_6th_avg['2015'] - country_6th_avg['2006'])
    return change
# example of test cases
assert type(answer_four()) == float, "Q4: You should return a single number!"

In [66]:
# example of test cases
assert type(answer_four()) == float, "Q4: You should return a single number!"

In [67]:
# space for proffesor tests


## Problem 5

What is the _mean energy supply per capita_?

_This function should return a single number._

In [68]:
def answer_five():
    # YOUR CODE HERE

    energy_supply_pc_mean = answer_one()['Energy Supply per Capita']
    return float(energy_supply_pc_mean.mean())
# example of test cases
assert type(answer_five()) == float, "Q5: You should return a single number!"

In [69]:
# example of test cases
assert type(answer_five()) == float, "Q5: You should return a single number!"

In [70]:
# space for proffesor tests

## Problem 6

What __country__ has the _maximum % Renewable_ and what is the _percentage_?

_This function should return a tuple with the name of the country and the percentage._

In [71]:
def answer_six():
    # YOUR CODE HERE
    max_renewable_country = answer_one()['% Renewable'].idxmax()
    max_renewable_percentage = answer_one()['% Renewable'].max()
    return (max_renewable_country, max_renewable_percentage)
# example of test cases
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"
assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"

In [72]:
# example of test cases
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"
assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"

In [73]:
# space for proffesor tests

## Problem 7

Create a new column that is the _ratio of Self-Citations to Total Citations_. 
What is the _maximum_ value for this new column, and what _country_ has the _highest ratio_?

_This function should return a tuple with the name of the country and the ratio._

In [74]:
def answer_seven():
    # YOUR CODE HERE
    rank_df = answer_one()
    rank_df['Citations Ratio'] = rank_df['Self-citations'] / rank_df['Citations']
    
    max_citations_country = rank_df['Citations Ratio'].idxmax()
    max_citations = rank_df['Citations Ratio'].max()
    
    return (max_citations_country, max_citations)
# example of test cases
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"
assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"

In [75]:
# example of test cases
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"
assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"

In [76]:
# space for proffesor tests

## Problem 8

Create a column that estimates the _population using Energy Supply_ and _Energy Supply per capita_. 
What is the _third_ most populous country according to this estimate?

_This function should return the name of the country._

In [77]:
def answer_eight():
    # YOUR CODE HERE
    rank_df = answer_one()
    rank_df["Stimed Population"] = rank_df["Energy Supply"] / rank_df["Energy Supply per Capita"]
    
    df2 = rank_df["Stimed Population"].sort_values(ascending=False)
    return df2.index[2]
# example of test cases
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"

In [78]:
# example of test cases
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"

In [79]:
# space for proffesor tests

## Problem 9

Create a column that estimates the _number of citable documents per person_. 

What is the correlation between the _number of citable documents per capita_ and the _energy supply per capita_? Use the `.corr()` method, (Pearson's correlation).

_This function should return a single number._

In [80]:
def answer_nine():
    # YOUR CODE HERE

    rank_df = answer_one()

    rank_df['Stimed population'] = rank_df['Energy Supply'] / rank_df['Energy Supply per Capita']

    rank_df['Citable documents per capita'] = rank_df['Citable documents'] / rank_df['Stimed population']

    correlation = rank_df['Citable documents per capita'].corr(rank_df['Energy Supply per Capita'])
    
    return correlation
# example of test cases
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"

In [81]:
# example of test cases
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"

In [82]:
# space for proffesor tests

## Problem 10

Create a new column with a $1$ if the _country's % Renewable_ value is at or above the median for all countries in the _top 15_, and a $0$ if the _country's % Renewable_ value is below the median.

_This function should return a series named `high_renew` whose index is the country name sorted in ascending order of rank._

In [83]:
def answer_ten():
    # YOUR CODE HERE
    
    rank_df = answer_one()
    rank_df['high_renew'] = rank_df['% Renewable'].apply(lambda ren: 1 if ren>=rank_df['% Renewable'].mean() else 0)
    return rank_df['high_renew']
# example of test cases
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"

In [84]:
# example of test cases
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"

In [85]:
# space for proffesor tests

## Problem 11

Use the following dictionary to group the _Countries by Continent_, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the _sum_, _mean_, and _std deviation_ for the estimated population of each country.

```python
ContinentDict  = {
    'China':'Asia', 
    'United States':'North America', 
    'Japan':'Asia', 
    'United Kingdom':'Europe', 
    'Russian Federation':'Europe', 
    'Canada':'North America', 
    'Germany':'Europe', 
    'India':'Asia',
    'France':'Europe', 
    'South Korea':'Asia', 
    'Italy':'Europe', 
    'Spain':'Europe', 
    'Iran':'Asia',
    'Australia':'Australia', 
    'Brazil':'South America'
}
```

_This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`_

In [86]:
ContinentDict  = {
    'China':'Asia', 
    'United States':'North America', 
    'Japan':'Asia', 
    'United Kingdom':'Europe', 
    'Russian Federation':'Europe', 
    'Canada':'North America', 
    'Germany':'Europe', 
    'India':'Asia',
    'France':'Europe', 
    'South Korea':'Asia', 
    'Italy':'Europe', 
    'Spain':'Europe', 
    'Iran':'Asia',
    'Australia':'Australia', 
    'Brazil':'South America'
}

def answer_eleven():
    # YOUR CODE HERE
    
    rank_df = answer_one()
    rank_df['continent'] = rank_df.apply(lambda con: ContinentDict[con.name], axis=1)
    rank_df['Stimed Population'] = rank_df['Energy Supply'] / rank_df['Energy Supply per Capita']

    group_data_frame = rank_df.groupby('continent', observed=False).agg({'Stimed Population': ['size', 'sum', 'mean', 'std']})
    
    return group_data_frame
# example of test cases
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"
assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"
assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"

In [87]:
# example of test cases
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"
assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"
assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"

In [88]:
# space for proffesor tests

## Problem 12

Cut (check `.cut()` from Pandas) _% Renewable_ into $5$ bins, but in the same column. Group _Top15_ by the _Continent_, as well as these new _% Renewable_ bins. How many countries are in each of these groups? You could check `.size()` as aggregation in group by.

_This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable` (in string datatype). Do not include groups with no countries._

In [89]:
def answer_twelve():
    # YOUR CODE HERE

    rank_df = answer_one()
    rank_df['% Renewable'] = pd.cut(rank_df['% Renewable'], bins=5)

    rank_df['% Renewable'] = rank_df['% Renewable'].astype(str)
    rank_df['continent'] = rank_df.apply(lambda con: ContinentDict[con.name], axis=1)
    
    group_data_frame = rank_df.groupby(['continent', '% Renewable'], observed=False).size()

    group_data_frame_without_countries = group_data_frame[group_data_frame > 0]
    return group_data_frame_without_countries
# example of test cases
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"
assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"

In [90]:
# example of test cases
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"
assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"

In [91]:
# space for proffesor tests

## Problem 13

Convert the _Population Estimate_ series to a string with thousands separator (using _dots_). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12.345.678,90

_This function should return a series `pop_est` whose index is the country name and whose values are the population estimate string_

In [92]:
def answer_thirteen():
    # YOUR CODE HERE
    rank_df = answer_one()
    rank_df["population"] = rank_df["Energy Supply"] / rank_df["Energy Supply per Capita"]
    pop_est = rank_df["population"].apply(lambda x: "{:,}".format(x))
    pop_est = pd.Series(pop_est, name='pop_est')
    pop_est = pop_est.apply(lambda x: re.sub(r"\.", ":", x))
    pop_est = pop_est.apply(lambda x: re.sub(r",", ".", x))
    pop_est = pop_est.apply(lambda x: re.sub(r":", ",", x))
    return pop_est
# example of test cases
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"
assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"

In [93]:
# example of test cases
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"
assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"

In [94]:
# space for proffesor tests